In [ ]:
import wntr
import pandas as pd

inp_file = '/mnt/data/home/zayd/Digital_twin_project/inp_networks/Anytown.inp'
wn= wntr.network.WaterNetworkModel(inp_file)

junction = []
for junc_name, junc in wn.junctions():
    junction.append([junc_name,junc.coordinates[0],junc.coordinates[1]])
df = pd.DataFrame(junction, columns=['name','X','Y'])
df.to_csv('coordinate.csv', index=False)

print('done')

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt


df = pd.read_csv('coordinate.csv')

coords = df[['X','Y']].values

k = 5
kmeans = KMeans(n_clusters= k,random_state = 42)
df['zone'] = kmeans.fit_predict(coords)

df.to_csv("neighbors.csv", index=False)

plt.scatter(df['X'],df['Y'], c = df['zone'],cmap = "Set1")
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Junction Zones")
plt.show()


In [ ]:
df['zone']

In [ ]:
import pandas as pd
import logging
from pathlib import Path

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler()]
)

# Paths
root_p = '/mnt/data/home/zayd/Digital_twin_project/machine_learning'
out_dir = Path(f"{root_p}/dataset/zones_v2")
out_dir.mkdir(parents=True, exist_ok=True)

# Load main demand data
demand = pd.read_csv(f"{root_p}/dataset/generated_v2.csv")

# Load junction → zone mapping (columns: ['name','zone'])
links = pd.read_csv("zones.csv")

# Group junctions by zone
zone_groups = links.groupby("zone")["name"].apply(list)

# Process each zone
for zone_id, junctions in zone_groups.items():
    logging.info(f"Processing Zone {zone_id} with {len(junctions)} junctions")

    # Start with time_id and day columns
    zone_df = demand[["time_id", "day"]].copy()

    # Merge all junctions for this zone
    for j in junctions:
        if j in demand.columns:
            zone_df = zone_df.merge(
                demand[["time_id", "day", j]].rename(columns={j: f"{j}_demand"}),
                on=["time_id", "day"],
                how="left"
            )

    # Save parquet per zone
    out_path = out_dir / f"zone_{zone_id}.parquet"
    zone_df.to_parquet(out_path, engine="pyarrow", index=False)
    logging.info(f"Saved Zone {zone_id} → {out_path} with shape {zone_df.shape}")


LTSM (single junction)


In [5]:
import os, time, pickle, json
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import mlflow
import mlflow.tensorflow

# -----------------------------
# Metrics
# -----------------------------
def smape(y_true, y_pred):
    denominator = np.abs(y_true) + np.abs(y_pred)
    denominator = np.where(denominator == 0, np.finfo(float).eps, denominator)
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / denominator)

def peak_error(y_true, y_pred, percentile=95):
    peak_value = np.percentile(y_true, percentile)
    peak_indices = y_true >= peak_value
    if np.sum(peak_indices) == 0:
        return np.nan
    return np.mean(np.abs(y_true[peak_indices] - y_pred[peak_indices]))

def create_sequences_by_scenario(df, seq_length, feature_cols, target_col):
    X, y, seq_scenario_ids = [], [], []
    for scenario_id, group in df.groupby('scenario_id'):
        group = group.sort_index()
        data = group[feature_cols + [target_col]].values
        if len(data) <= seq_length: continue
        for i in range(seq_length, len(data)):
            X.append(data[i-seq_length:i, :-1])
            y.append(data[i, -1])
            seq_scenario_ids.append(scenario_id)
    return np.array(X), np.array(y), np.array(seq_scenario_ids)

# -----------------------------
# Paths & parameters
# -----------------------------
root = '/mnt/data/home/zayd/Digital_twin_project/machine_learning/dataset/Ctown/junctions/'
version = '0.1.0'
MODEL_DIR = f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/LSTM_{version}'
os.makedirs(MODEL_DIR, exist_ok=True)

seq_length = 23
epochs = 50
batch_size = 32
lag_steps = [1,2,3]
rolling_windows = [3,6,12]

mlflow.set_experiment("Digital_Twin_Experiments")
global_y_true, global_y_pred = [], []
feature_and_target = {}

# -----------------------------
# Parent MLflow run
# -----------------------------
with mlflow.start_run(run_name=f"LSTM_{version}"):

    mlflow.log_param("seq_length", seq_length)
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("batch_size", batch_size)

    for filename in os.listdir(root)[:10]:
        df = pd.read_parquet(os.path.join(root, filename))
        df = df.sort_values(by=["scenario_id", "time_id"]).reset_index(drop=True)
        junction_cols = [col for col in df.columns if col.startswith('J')]

        # Convert from m³/s to L/s
        df[junction_cols] = df[junction_cols] * 1000
        junction = os.path.splitext(filename)[0]

        with mlflow.start_run(run_name=f"{junction}", nested=True):
            raw_cols = df.columns.tolist()
            target = junction
            features = [c for c in df.columns if c != target]
            feature_and_target[junction] = {"target": target, "features": features, "raw_cols": raw_cols}

            # -----------------------------
            # Lag and rolling features
            # -----------------------------
            for lag in lag_steps:
                df[f'{junction}_lag{lag}'] = df[junction].shift(lag)
            for w in rolling_windows:
                df[f'{junction}_rollmean{w}'] = df[junction].rolling(window=w, min_periods=1).mean()
            df = df.fillna(0)

            # -----------------------------
            # Train/test split
            # -----------------------------
            scenario_ids = df['scenario_id'].unique()
            split = int(0.8*len(scenario_ids))
            train_df = df[df['scenario_id'].isin(scenario_ids[:split])]
            test_df = df[df['scenario_id'].isin(scenario_ids[split:])]

            # -----------------------------
            # Scaling
            # -----------------------------
            feature_scaler = MinMaxScaler().fit(train_df[features])
            target_scaler = MinMaxScaler().fit(train_df[[target]])

            X_train_scaled = feature_scaler.transform(train_df[features])
            y_train_scaled = target_scaler.transform(train_df[[target]])
            X_test_scaled = feature_scaler.transform(test_df[features])
            y_test_scaled = target_scaler.transform(test_df[[target]])

            # Save scalers
            with open(os.path.join(MODEL_DIR,f"{junction}_feature_scaler.save"),'wb') as f: pickle.dump(feature_scaler,f)
            with open(os.path.join(MODEL_DIR,f"{junction}_target_scaler.save"),'wb') as f: pickle.dump(target_scaler,f)

            # -----------------------------
            # Create sequences
            # -----------------------------
            X_train, y_train, _ = create_sequences_by_scenario(train_df, seq_length, features, target)
            X_test, y_test, _ = create_sequences_by_scenario(test_df, seq_length, features, target)
            n_features = X_train.shape[2]
            n_targets = 1
            mlflow.log_param(f"n_features_{junction}", n_features)

            # -----------------------------
            # Build and train LSTM
            # -----------------------------
            model = tf.keras.Sequential([
                tf.keras.layers.LSTM(50, activation='relu', input_shape=(seq_length,n_features)),
                tf.keras.layers.Dense(n_targets)
            ])
            model.compile(optimizer='adam', loss='mse')

            start_time = time.time()
            history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))
            elapsed_sec = time.time() - start_time
            mlflow.log_param(f"{junction}_training_time_sec", elapsed_sec)

            # Save model
            model_path = os.path.join(MODEL_DIR,f"{junction}.keras")
            model.save(model_path)
            mlflow.tensorflow.log_model(model, artifact_path=junction, registered_model_name=junction,
                                        input_example=np.random.rand(1, seq_length, n_features))

            # -----------------------------
            # Predictions & inverse scaling
            # -----------------------------
            y_pred = model.predict(X_test)
            y_test_inv = target_scaler.inverse_transform(y_test.reshape(-1,1))
            y_pred_inv = target_scaler.inverse_transform(y_pred.reshape(-1,1))
            global_y_true.append(y_test_inv)
            global_y_pred.append(y_pred_inv)

            # Metrics
            mae = mean_absolute_error(y_test_inv, y_pred_inv)
            rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
            smape_val = smape(y_test_inv, y_pred_inv)
            r2 = r2_score(y_test_inv, y_pred_inv)
            peak_err = peak_error(y_test_inv, y_pred_inv)

            mlflow.log_params({
                "algo": f"LSTM_{version}",
                "junction": junction,
            }
            )

            mlflow.log_metrics({
                "MAE": mae, "RMSE": rmse, "SMAPE": smape_val, "R2_score": r2, "peak_error": peak_err
            })

            # -----------------------------
            # Plot loss
            # -----------------------------
            plt.figure(figsize=(10,6))
            plt.plot(history.history['loss'], label='train_loss')
            plt.plot(history.history['val_loss'], label='val_loss')
            plt.legend(); plt.title(f'{junction} Loss Curve')
            loss_plot_path = os.path.join(MODEL_DIR,f"{junction}_loss_curve.png")
            plt.savefig(loss_plot_path); plt.close()
            mlflow.log_artifact(loss_plot_path, artifact_path=f"{junction}/plots")

            # -----------------------------
            # Plot predictions
            # -----------------------------
            plt.figure(figsize=(14,7))
            plt.plot(y_test_inv, label='Actual', color='orange')
            plt.plot(y_pred_inv, label='Predicted', color='green')
            plt.title(f'{junction} Forecast')
            plt.xlabel('Time step'); plt.ylabel('Value'); plt.legend()
            forecast_plot_path = os.path.join(MODEL_DIR,f"{junction}_forecast.png")
            plt.savefig(forecast_plot_path); plt.close()
            mlflow.log_artifact(forecast_plot_path, artifact_path=f"{junction}/plots")

    # -----------------------------
    # Global metrics
    # -----------------------------
    all_y_true = np.vstack(global_y_true)
    all_y_pred = np.vstack(global_y_pred)
    global_metrics = {
        'MAE': mean_absolute_error(all_y_true, all_y_pred),
        'RMSE': np.sqrt(mean_squared_error(all_y_true, all_y_pred)),
        'SMAPE': smape(all_y_true, all_y_pred),
        'R2_score': r2_score(all_y_true, all_y_pred),
        'peak_error': peak_error(all_y_true, all_y_pred)
    }
    mlflow.log_metrics(global_metrics)
    with open(os.path.join(MODEL_DIR,'feature_and_target.json'),'w') as f: json.dump(feature_and_target,f,indent=4)

print("✅ LSTM training with parent MLflow run, nested runs, and plots completed")


2025-09-14 13:02:36.971237: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-14 13:02:36.978457: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-14 13:02:37.002239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757851357.046008   11861 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757851357.061914   11861 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757851357.092176   11861 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

Epoch 1/50


2025-09-14 13:02:43.198850: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/zayd/Desktop/Digital_twin_project/digitaltwin/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - loss: 628093120.0000 - val_loss: 26941644.0000
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 4388324.5000 - val_loss: 559751.5000
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 503659.9688 - val_loss: 586804.0000
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 324005.0000 - val_loss: 660382.3750
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 256895.7812 - val_loss: 606876.9375
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 187354.0625 - val_loss: 620487.7500
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 91983.3984 - val_loss: 431693.5312
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 74122.4609 - val_loss: 428535.6250
Epoch 9/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 76964.9141 - val_loss: 441944.0000
Epoch 10/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 57136.5234 - val_loss: 353659.7500
Epoch 11/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss

2025/09/14 13:03:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


...

25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 994969152.0000 - val_loss: 43075736.0000
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 57661952.0000 - val_loss: 26266440.0000
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 19810026.0000 - val_loss: 98320872.0000
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 76301288.0000 - val_loss: 34240844.0000
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 22900442.0000 - val_loss: 58223492.0000
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 58758792.0000 - val_loss: 41306348.0000
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 42111864.0000 - val_loss: 20445530.0000
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 56654772.0000 - val_loss: 1823742.2500
Epoch 9/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 1424854.6250 - val_loss: 62556.2109
Epoch 10/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 7442575.0000 - val_loss: 22502000.0000
Epoch 11/50
25/25 ━━━━━━━

2025/09/14 13:09:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step


Registered model 'J498' already exists. Creating a new version of this model...
Created version '2' of model 'J498'.


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step 
✅ LSTM training with parent MLflow run, nested runs, and plots completed


LSTM (zones)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import matplotlib.pyplot as plt
import pickle
import os
import mlflow
import time
import json

# -----------------------------
# Helper functions
# -----------------------------
def create_sequences_multioutput(data, seq_length, target_indices):
    """
    data: np.array of shape (n_samples, n_features)
    seq_length: length of input sequences
    target_indices: list of column indices for targets in data
    Returns: X, y arrays for LSTM
    """
    X, y = [], []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i, :])           # all columns (features + targets) as input
        y.append(data[i, target_indices])           # select only target columns
    return np.array(X), np.array(y)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.where(y_true == 0, np.finfo(float).eps, y_true)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def smape(y_true, y_pred):
    denominator = np.where((np.abs(y_true) + np.abs(y_pred)) == 0, np.finfo(float).eps, np.abs(y_true) + np.abs(y_pred))
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / denominator)

def peak_error(y_true, y_pred, percentile=95):
    peak_value = np.percentile(y_true, percentile)
    peak_indices = y_true >= peak_value
    if np.sum(peak_indices) == 0: return np.nan
    return np.mean(np.abs(y_true[peak_indices] - y_pred[peak_indices]))

# -----------------------------
# Config
# -----------------------------
version = '1.0.1'
seq_length = 12
lag_steps = [1,2,3]
rolling_windows = [3,6,12]
root = '/mnt/data/home/zayd/Digital_twin_project/machine_learning/dataset/zones_v2/'
trained_dir = f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/LSTM_{version}/'
os.makedirs(trained_dir, exist_ok=True)
trained_dir_list = os.listdir(trained_dir)
metrics_df = pd.DataFrame(columns=['zone','MAE','RMSE','MAPE','SMAPE','R2','Peak_Error'])
mlflow.set_tracking_uri("file:model_trained/mlruns")
mlflow.set_experiment("Digital_Twin_LSTM")

feature_and_target = {}

# -----------------------------
# Process zones
# -----------------------------
with mlflow.start_run(run_name=f"LSTM_{version}"):

    for filename in os.listdir(root):
        zone_name = os.path.splitext(filename)[0]
        if f'{zone_name}.keras' in trained_dir_list:
            print(f"Zone {zone_name} already trained, skipping")
            continue
        with mlflow.start_run(run_name=f"Zone_{zone_name}", nested=True):

            df = pd.read_parquet(os.path.join(root, filename))
            junctions = [col for col in df.columns if 'demand' in col]

            # -----------------------------
            # Lag & rolling features
            # -----------------------------
            for lag in lag_steps:
                for j in junctions:
                    df[f'{j}_lag{lag}'] = df[j].shift(lag)

            for w in rolling_windows:
                for j in junctions:
                    df[f'{j}_rollmean{w}'] = df[j].rolling(window=w, min_periods=1).mean()

            df = df.fillna(0)
            target_cols = junctions.copy()
            y_cols_idx = [df.columns.get_loc(c) for c in target_cols]
            features = [col for col in df.columns if col not in target_cols]

            # -----------------------------
            # Train/Test split
            # -----------------------------
            df = df.sort_values(['day','time_id'])
            split_idx = int(0.8*len(df))
            train_df, test_df = df.iloc[:split_idx], df.iloc[split_idx:]

            # Scale features
            feature_scaler = MinMaxScaler()
            target_scaler = MinMaxScaler()
            X_train_scaled = feature_scaler.fit_transform(train_df[features])
            y_train_scaled = target_scaler.fit_transform(train_df[target_cols])
            X_test_scaled = feature_scaler.transform(test_df[features])
            y_test_scaled = target_scaler.transform(test_df[target_cols])

            train_scaled_df = pd.DataFrame(np.hstack((X_train_scaled, y_train_scaled)), columns=features+target_cols)
            test_scaled_df = pd.DataFrame(np.hstack((X_test_scaled, y_test_scaled)), columns=features+target_cols)

            # Save scalers
            with open(os.path.join(trained_dir, f'{zone_name}_feature_scaler.save'), 'wb') as f:
                pickle.dump(feature_scaler, f)
            with open(os.path.join(trained_dir, f'{zone_name}_target_scaler.save'), 'wb') as f:
                pickle.dump(target_scaler, f)

            # -----------------------------
            # Create sequences
            # -----------------------------
            target_indices = [train_scaled_df.columns.get_loc(c) for c in target_cols]

            X_train, y_train = create_sequences_multioutput(train_scaled_df.values, seq_length, target_indices)
            X_test, y_test = create_sequences_multioutput(test_scaled_df.values, seq_length, target_indices)
            n_features = X_train.shape[2]
            n_targets = y_train.shape[1] if len(y_train.shape) > 1 else 1

            feature_and_target[zone_name] = {
                "target": target_cols,
                "features": features,
                "raw_cols": df.columns.tolist()
            }

            # Log parameters to MLflow
            mlflow.log_param("zone", zone_name)
            mlflow.log_param("seq_length", seq_length)
            mlflow.log_param("n_features", n_features)
            mlflow.log_param("n_targets", n_targets)
            mlflow.log_param("lag_steps", lag_steps)
            mlflow.log_param("rolling_windows", rolling_windows)
            mlflow.log_param("epochs", 50)
            mlflow.log_param("batch_size", 32)

            # -----------------------------
            # Build & train multi-output LSTM
            # -----------------------------
            model = tf.keras.Sequential([
                tf.keras.layers.LSTM(50, activation='relu', input_shape=(seq_length, n_features)),
                tf.keras.layers.Dense(n_targets)
            ])
            model.compile(optimizer='adam', loss='mse')

            start = time.time()
            history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))
            elapsed_sec = time.time() - start
            minutes, rem = divmod(elapsed_sec, 60)
            seconds, milliseconds = divmod(rem, 1)
            milliseconds = int(milliseconds * 1000)
            formatted_time = f"{int(minutes)}m:{int(seconds)}s:{milliseconds}ms"
            mlflow.log_param(f"{zone_name}_training_time", formatted_time)

            # Save model
            model.save(os.path.join(trained_dir, f"{zone_name}.keras"))
            with open(os.path.join(trained_dir, f"{zone_name}_feature_and_target.json"), "w") as f:
                json.dump(feature_and_target, f, indent=4)

            # Log model summary
            model_summary = []
            model.summary(print_fn=lambda x: model_summary.append(x))
            mlflow.log_text("\n".join(model_summary), artifact_file=f"{zone_name}/model_summary.txt")
            mlflow.tensorflow.log_model(model, f"model_{zone_name}")

            
            # Predict & inverse scale
            y_pred = model.predict(X_test)
            y_test_inv = target_scaler.inverse_transform(y_test)
            y_pred_inv = target_scaler.inverse_transform(y_pred)

            # Evaluate
            mae = mean_absolute_error(y_test_inv, y_pred_inv)
            rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
            mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)
            smape_val = smape(y_test_inv, y_pred_inv)
            r2 = r2_score(y_test_inv, y_pred_inv)
            peak_err = peak_error(y_test_inv, y_pred_inv)

            metrics_df = pd.concat([metrics_df, pd.DataFrame({
                'zone':[zone_name],'MAE':[mae],'RMSE':[rmse],'MAPE':[mape],'SMAPE':[smape_val],'R2':[r2],'Peak_Error':[peak_err]
            })], ignore_index=True)

            # Log metrics
            mlflow.log_metrics({
                "MAE": mae,
                "RMSE": rmse,
                "MAPE": mape,
                "SMAPE": smape_val,
                "R2": r2,
                "Peak_Error": peak_err
            })
            print(f"✅ Zone {zone_name}: MAE={mae:.3f}, RMSE={rmse:.3f}, R2={r2:.3f}")

            # Save scalers in MLflow
            mlflow.log_artifact(os.path.join(trained_dir, f'{zone_name}_feature_scaler.save'), artifact_path=f"{zone_name}/scalers")
            mlflow.log_artifact(os.path.join(trained_dir, f'{zone_name}_target_scaler.save'), artifact_path=f"{zone_name}/scalers")

            input_example = np.random.rand(1, seq_length, n_features)
            mlflow.tensorflow.log_model(model, artifact_path=zone_name, registered_model_name=zone_name, input_example=input_example)

            # Plot loss
            plt.figure()
            plt.plot(history.history['loss'], label='train_loss')
            plt.plot(history.history['val_loss'], label='val_loss')
            plt.legend(); plt.title(f'Loss over epochs - {zone_name}')
            plt.savefig(os.path.join(trained_dir, f"{zone_name}_loss_curve.png"))
            mlflow.log_artifact(os.path.join(trained_dir, f"{zone_name}_loss_curve.png"), artifact_path=f"{zone_name}/plots")
            plt.close()

metrics_df.set_index('zone', inplace=True)
print(metrics_df)


LightGBM (single junction)


In [7]:
import os, time, pickle, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import mlflow

# -----------------------------
# Metrics
# -----------------------------
def smape(y_true, y_pred):
    denominator = np.abs(y_true) + np.abs(y_pred)
    denominator = np.where(denominator == 0, np.finfo(float).eps, denominator)
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / denominator)

def peak_error(y_true, y_pred, percentile=95):
    peak_value = np.percentile(y_true, percentile)
    peak_indices = y_true >= peak_value
    if np.sum(peak_indices) == 0:
        return np.nan
    return np.mean(np.abs(y_true[peak_indices] - y_pred[peak_indices]))

# -----------------------------
# Paths and parameters
# -----------------------------
root = '/mnt/data/home/zayd/Digital_twin_project/machine_learning/dataset/Ctown/junctions/'
version = '0.1.0'
MODEL_DIR = f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/LightGBM_{version}'
os.makedirs(f'{MODEL_DIR}/scalers', exist_ok=True)

lag_steps = [1,2,3]
rolling_windows = [3,6,12]

mlflow.set_experiment("Digital_Twin_Experiments")
global_y_true, global_y_pred = [], []
feature_and_target = {}

# -----------------------------
# Parent MLflow run
# -----------------------------
with mlflow.start_run(run_name=f"LightGBM_{version}"):  
    for filename in os.listdir(root)[:10]:
        df = pd.read_parquet(os.path.join(root, filename))
        df = df.sort_values(by=["scenario_id", "time_id"]).reset_index(drop=True)
        junction_cols = [col for col in df.columns if col.startswith('J')]

        # Convert from m³/s to L/s
        df[junction_cols] = df[junction_cols] * 1000
        junction = os.path.splitext(filename)[0]

        with mlflow.start_run(run_name=f"{junction}", nested=True):
            # -----------------------------
            # Feature engineering
            # -----------------------------
            for lag in lag_steps: df[f'{junction}_lag{lag}'] = df[junction].shift(lag)
            for w in rolling_windows: df[f'{junction}_rollmean{w}'] = df[junction].rolling(window=w, min_periods=1).mean()
            df = df.fillna(0)

            features = [c for c in df.columns if c != junction]
            target = junction
            feature_and_target[junction] = {"target": target, "features": features, "lags": lag_steps, "rolling": rolling_windows}

            # -----------------------------
            # Scaling
            # -----------------------------
            feature_scaler = MinMaxScaler().fit(df[features])
            target_scaler = MinMaxScaler().fit(df[[target]])
            df[features] = feature_scaler.transform(df[features])
            df[[target]] = target_scaler.transform(df[[target]])

            # Save scalers
            with open(f'{MODEL_DIR}/scalers/{junction}_feature_scaler.save', 'wb') as f: pickle.dump(feature_scaler, f)
            with open(f'{MODEL_DIR}/scalers/{junction}_target_scaler.save', 'wb') as f: pickle.dump(target_scaler, f)

            # -----------------------------
            # Train/test split
            # -----------------------------
            scenario_ids = df['scenario_id'].unique()
            split = int(0.8 * len(scenario_ids))
            train_df = df[df['scenario_id'].isin(scenario_ids[:split])]
            test_df = df[df['scenario_id'].isin(scenario_ids[split:])]
            X_train, y_train = train_df[features], train_df[target]
            X_test, y_test = test_df[features], test_df[target]

            # -----------------------------
            # Train LightGBM
            # -----------------------------
            train_data = lgb.Dataset(X_train, label=y_train)
            valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
            params = {'objective':'regression','metric':'rmse','learning_rate':0.01,'num_leaves':31,
                      'feature_fraction':0.8,'bagging_fraction':0.8,'bagging_freq':5,'verbose':-1}
            start_time = time.time()
            model = lgb.train(params, train_data, num_boost_round=5000, valid_sets=[train_data, valid_data],
                              callbacks=[lgb.early_stopping(100), lgb.log_evaluation(period=100)])
            elapsed_sec = time.time() - start_time

            # -----------------------------
            # Predictions & metrics
            # -----------------------------
            y_pred = model.predict(X_test, num_iteration=model.best_iteration)
            y_test_inv = target_scaler.inverse_transform(y_test.values.reshape(-1,1))
            y_pred_inv = target_scaler.inverse_transform(y_pred.reshape(-1,1))
            global_y_true.append(y_test_inv)
            global_y_pred.append(y_pred_inv)

            # Log parameters & metrics for junction
            mlflow.log_params({
                "algo": f"LightGBM_{version}",
                "junction": junction,
                "window": rolling_windows,
                "stride": lag_steps,
                "num_leaves": 31,
                "learning_rate": 0.01,
                "feature_fraction": 0.8,
                "bagging_fraction": 0.8,
                "bagging_freq": 5
            })
            mlflow.log_metric("training_time_sec", elapsed_sec)
            mlflow.log_metric("MAE", mean_absolute_error(y_test_inv, y_pred_inv))
            mlflow.log_metric("RMSE", np.sqrt(mean_squared_error(y_test_inv, y_pred_inv)))
            mlflow.log_metric("SMAPE", smape(y_test_inv, y_pred_inv))
            mlflow.log_metric("R2_score", r2_score(y_test_inv, y_pred_inv))
            mlflow.log_metric("peak_error", peak_error(y_test_inv, y_pred_inv))
            

            # -----------------------------
            # Save model artifact
            # -----------------------------
            model_path = os.path.join(MODEL_DIR, f"{junction}.txt")
            model.save_model(model_path)
            mlflow.log_artifact(model_path)

            # -----------------------------
            # Plot predictions
            # -----------------------------
            plt.figure(figsize=(12,6))
            plt.plot(y_test_inv[:100], label='Actual', color='orange')
            plt.plot(y_pred_inv[:100], label='Predicted', color='green')
            plt.xlabel('Time step')
            plt.ylabel('Value')
            plt.title(f'{junction} LightGBM Forecast')
            plt.legend()
            plot_path = os.path.join(MODEL_DIR, f"{junction}_forecast.png")
            plt.savefig(plot_path)
            plt.close()
            mlflow.log_artifact(plot_path)

    # -----------------------------
    # Log global metrics for parent run
    # -----------------------------
    all_y_true = np.vstack(global_y_true)
    all_y_pred = np.vstack(global_y_pred)
    global_metrics = {
        'MAE': mean_absolute_error(all_y_true, all_y_pred),
        'RMSE': np.sqrt(mean_squared_error(all_y_true, all_y_pred)),
        'SMAPE': smape(all_y_true, all_y_pred),
        'R2_score': r2_score(all_y_true, all_y_pred),
        'peak_error': peak_error(all_y_true, all_y_pred)
    }
    mlflow.log_metrics(global_metrics)

    # Save feature/target mapping
    with open(f'{MODEL_DIR}/feature_and_target.json', 'w') as f: json.dump(feature_and_target, f, indent=4)

print("✅ LightGBM training with parent MLflow run and plots completed")


Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.144858	valid_1's rmse: 0.148556
[200]	training's rmse: 0.0900521	valid_1's rmse: 0.0954436
[300]	training's rmse: 0.0623927	valid_1's rmse: 0.068747
[400]	training's rmse: 0.0454983	valid_1's rmse: 0.0523205
[500]	training's rmse: 0.0351123	valid_1's rmse: 0.0421116
[600]	training's rmse: 0.0289784	valid_1's rmse: 0.036022
[700]	training's rmse: 0.0250722	valid_1's rmse: 0.0321021
[800]	training's rmse: 0.0223724	valid_1's rmse: 0.0293711
[900]	training's rmse: 0.0203823	valid_1's rmse: 0.0273637
[1000]	training's rmse: 0.0190099	valid_1's rmse: 0.0260134
[1100]	training's rmse: 0.0180049	valid_1's rmse: 0.0251369
[1200]	training's rmse: 0.0171381	valid_1's rmse: 0.0243961
[1300]	training's rmse: 0.0164697	valid_1's rmse: 0.0238454
[1400]	training's rmse: 0.0158987	valid_1's rmse: 0.0234491
[1500]	training's rmse: 0.0154012	valid_1's rmse: 0.0231137
[1600]	training's rmse: 0.0149344	valid_1's rmse: 0

LightGBM (hyperparameter tuning + expanding-window / forward-chaining CV)


In [ ]:
import os, time, pickle, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import mlflow
import optuna

# -----------------------------
# Metrics
# -----------------------------
def smape(y_true, y_pred):
    denominator = np.abs(y_true) + np.abs(y_pred)
    denominator = np.where(denominator == 0, np.finfo(float).eps, denominator)
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / denominator)

def peak_error(y_true, y_pred, percentile=95):
    peak_value = np.percentile(y_true, percentile)
    peak_indices = y_true >= peak_value
    if np.sum(peak_indices) == 0:
        return np.nan
    return np.mean(np.abs(y_true[peak_indices] - y_pred[peak_indices]))

# -----------------------------
# Paths and parameters
# -----------------------------
root = '/mnt/data/home/zayd/Digital_twin_project/machine_learning/dataset/Ctown/junctions/'
version = '0.0.5_hyperparams'
MODEL_DIR = f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/LightGBM_{version}'
os.makedirs(f'{MODEL_DIR}/scalers', exist_ok=True)

lag_steps = [1,2,3]
rolling_windows = [3,6,12]

mlflow.set_experiment("Digital_Twin_Experiments")
global_y_true, global_y_pred = [], []
feature_and_target = {}

# -----------------------------
# Parent MLflow run
# -----------------------------
with mlflow.start_run(run_name=f"LightGBM_{version}"):

    for filename in os.listdir(root)[:10]:
        df = pd.read_parquet(os.path.join(root, filename))
        df = df.sort_values(by=["scenario_id", "time_id"]).reset_index(drop=True)
        junction = os.path.splitext(filename)[0]

        with mlflow.start_run(run_name=f"{junction}", nested=True):

            # -----------------------------
            # Feature engineering
            # -----------------------------
            for lag in lag_steps: df[f'{junction}_lag{lag}'] = df[junction].shift(lag)
            for w in rolling_windows: df[f'{junction}_rollmean{w}'] = df[junction].rolling(window=w, min_periods=1).mean()
            df = df.fillna(0)

            features = [c for c in df.columns if c != junction]
            target = junction
            feature_and_target[junction] = {"target": target, "features": features, "lags": lag_steps, "rolling": rolling_windows}

            # -----------------------------
            # Scaling
            # -----------------------------
            feature_scaler = MinMaxScaler().fit(df[features])
            target_scaler = MinMaxScaler().fit(df[[target]])
            df[features] = feature_scaler.transform(df[features])
            df[[target]] = target_scaler.transform(df[[target]])

            with open(f'{MODEL_DIR}/scalers/{junction}_feature_scaler.save', 'wb') as f: pickle.dump(feature_scaler, f)
            with open(f'{MODEL_DIR}/scalers/{junction}_target_scaler.save', 'wb') as f: pickle.dump(target_scaler, f)

            # -----------------------------
            # Prepare 30-day block CV splits for Optuna
            # -----------------------------
            scenario_ids = df['scenario_id'].unique()
            block_size = 30
            train_val_splits = []
            start = 0
            num_days = len(scenario_ids)

            while (start + 2*block_size) <= (num_days - 1):  # reserve last block for holdout
                train_days = scenario_ids[start : start + block_size]
                val_days = scenario_ids[start + block_size : start + 2*block_size]
                train_idx = df[df['scenario_id'].isin(train_days)].index
                val_idx = df[df['scenario_id'].isin(val_days)].index
                train_val_splits.append((train_idx, val_idx))
                start += block_size

            X = df[features]
            y = df[target].values

            # -----------------------------
            # Optuna hyperparameter tuning
            # -----------------------------
            def objective(trial):
                params = {
                    'objective': 'regression',
                    'metric': 'rmse',
                    'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
                    'num_leaves': trial.suggest_int('num_leaves', 16, 128),
                    'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 1.0),
                    'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.6, 1.0),
                    'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
                    'min_data_in_leaf': 5,
                    'min_data_in_bin': 5,
                    'verbose': -1
                }
                val_errors = []
                for train_idx, val_idx in train_val_splits:
                    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
                    y_train, y_val = y[train_idx], y[val_idx]
                    train_data = lgb.Dataset(X_train, label=y_train)
                    valid_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
                    model = lgb.train(params, train_data, num_boost_round=5000,
                                      valid_sets=[valid_data],
                                      callbacks=[lgb.early_stopping(100), lgb.log_evaluation(period=0)])
                    y_pred = model.predict(X_val, num_iteration=model.best_iteration)
                    y_pred_inv = target_scaler.inverse_transform(y_pred.reshape(-1,1))
                    y_val_inv = target_scaler.inverse_transform(y_val.reshape(-1,1))
                    val_errors.append(mean_squared_error(y_val_inv, y_pred_inv))
                return np.mean(val_errors)

            study = optuna.create_study(direction='minimize')
            study.optimize(objective, n_trials=50)
            best_params = study.best_params

            # -----------------------------
            # Log parameters
            # -----------------------------
            mlflow.log_params({
                "algo": f"LightGBM_{version}",
                "junction": junction,
                "window": rolling_windows,
                "stride": lag_steps
            })
            mlflow.log_params(best_params)

            # -----------------------------
            # Final training with last 30-day holdout
            # -----------------------------
            final_val_days = scenario_ids[-block_size:]
            train_days = scenario_ids[:-block_size]

            train_idx = df[df['scenario_id'].isin(train_days)].index
            val_idx = df[df['scenario_id'].isin(final_val_days)].index

            X_train, y_train = df.loc[train_idx, features], df.loc[train_idx, target]
            X_val, y_val = df.loc[val_idx, features], df.loc[val_idx, target]

            train_data = lgb.Dataset(X_train, label=y_train)
            valid_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

            start_time = time.time()
            best_model = lgb.train(best_params, train_data, num_boost_round=5000,
                                   valid_sets=[train_data, valid_data],
                                   callbacks=[lgb.early_stopping(100), lgb.log_evaluation(period=100)])
            elapsed_sec = time.time() - start_time

            # -----------------------------
            # Predictions & metrics
            # -----------------------------
            y_pred = best_model.predict(X_val, num_iteration=best_model.best_iteration)
            y_val_inv = target_scaler.inverse_transform(y_val.values.reshape(-1,1))
            y_pred_inv = target_scaler.inverse_transform(y_pred.reshape(-1,1))

            global_y_true.append(y_val_inv)
            global_y_pred.append(y_pred_inv)

            mlflow.log_metric("training_time_sec", elapsed_sec)
            mlflow.log_metric("MAE", mean_absolute_error(y_val_inv, y_pred_inv))
            mlflow.log_metric("RMSE", np.sqrt(mean_squared_error(y_val_inv, y_pred_inv)))
            mlflow.log_metric("SMAPE", smape(y_val_inv, y_pred_inv))
            mlflow.log_metric("R2_score", r2_score(y_val_inv, y_pred_inv))
            mlflow.log_metric("peak_error", peak_error(y_val_inv, y_pred_inv))

            # -----------------------------
            # Save model artifact
            # -----------------------------
            model_path = os.path.join(MODEL_DIR, f"{junction}_best_final.txt")
            best_model.save_model(model_path)
            mlflow.log_artifact(model_path)

            # -----------------------------
            # Plot predictions
            # -----------------------------
            plt.figure(figsize=(12,6))
            plt.plot(y_val_inv[:100], label='Actual', color='orange')
            plt.plot(y_pred_inv[:100], label='Predicted', color='green')
            plt.xlabel('Time step')
            plt.ylabel('Value')
            plt.title(f'{junction} LightGBM Forecast')
            plt.legend()
            plot_path = os.path.join(MODEL_DIR, f"{junction}_forecast.png")
            plt.savefig(plot_path)
            plt.close()
            mlflow.log_artifact(plot_path)

    # -----------------------------
    # Log global metrics
    # -----------------------------
    all_y_true = np.vstack(global_y_true)
    all_y_pred = np.vstack(global_y_pred)
    global_metrics = {
        'MAE': mean_absolute_error(all_y_true, all_y_pred),
        'RMSE': np.sqrt(mean_squared_error(all_y_true, all_y_pred)),
        'SMAPE': smape(all_y_true, all_y_pred),
        'R2_score': r2_score(all_y_true, all_y_pred),
        'peak_error': peak_error(all_y_true, all_y_pred)
    }
    mlflow.log_metrics(global_metrics)

    # Save feature/target mapping
    with open(f'{MODEL_DIR}/feature_and_target.json', 'w') as f: json.dump(feature_and_target, f, indent=4)

print("✅ LightGBM training with 30-day block CV, Optuna tuning, last-block holdout, and MLflow logging completed")


LightGBM (zones)

In [ ]:
# -----------------------------
# Paths and parameters
# -----------------------------
root = '/mnt/data/home/zayd/Digital_twin_project/machine_learning/dataset/zones_v2/'
version = '1.0.1'
os.makedirs(f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/LightGBM_{version}', exist_ok=True)
mlflow.set_tracking_uri("file:model_trained/mlruns")

metrics_df = pd.DataFrame(columns=['zone', 'MAE', 'RMSE', 'MAPE', 'SMAPE', 'R2', 'Peak_Error'])
feature_and_target = {}

lag_steps = [1,2,3]
rolling_windows = [3,6,12]  # moving averages (in timesteps)

# lists to store global predictions
global_y_true, global_y_pred = [], []

with mlflow.start_run(run_name=f"LightGBM_{version}"):

    for filename in os.listdir(root)[:10]:
        df = pd.read_parquet(os.path.join(root, filename))
        zone = os.path.splitext(filename)[0]

        # -----------------------------
        # Lag features
        # -----------------------------
        for lag in lag_steps:
            df[f'{zone}_lag{lag}'] = df[zone].shift(lag)

        # -----------------------------
        # Moving average features
        # -----------------------------
        for w in rolling_windows:
            df[f'{zone}_rollmean{w}'] = df[zone].rolling(window=w, min_periods=1).mean()

        df = df.fillna(0)

        features = [col for col in df.columns if col != zone]
        target = zone
        feature_and_target[zone] = {"target": target, "features": features, 'lags':lag_steps ,'rolling':rolling_windows}

        # -----------------------------
        # Scaling
        # -----------------------------
        feature_scaler = MinMaxScaler()
        target_scaler = MinMaxScaler()
        df[features] = feature_scaler.fit_transform(df[features])
        df[[target]] = target_scaler.fit_transform(df[[target]])

        # Save scalers
        os.makedirs(f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/LightGBM_{version}/scalers', exist_ok=True)
        with open(f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/LightGBM_{version}/scalers/{zone}_feature_scaler.save', 'wb') as f:
            pickle.dump(feature_scaler, f)
        with open(f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/LightGBM_{version}/scalers/{zone}_target_scaler.save', 'wb') as f:
            pickle.dump(target_scaler, f)

        # -----------------------------
        # Train/test split by time
        # -----------------------------
        time_ids = df['time'].unique()
        split = int(0.8 * len(time_ids))
        train_times = time_ids[:split]
        test_times = time_ids[split:]
        train_df = df[df['time'].isin(train_times)]
        test_df = df[df['time'].isin(test_times)]

        X_train, y_train = train_df[features], train_df[target]
        X_test, y_test = test_df[features], test_df[target]

        # -----------------------------
        # Train LightGBM
        # -----------------------------
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
        params = {
            'objective': 'regression',
            'metric': 'rmse',
            'learning_rate': 0.01,
            'num_leaves': 31,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': -1
        }
        start_time = time.time()
        model = lgb.train(
            params,
            train_data,
            num_boost_round=5000,
            valid_sets=[train_data, valid_data],
            callbacks=[
                lgb.early_stopping(stopping_rounds=100),
                lgb.log_evaluation(period=100)
            ]
        )

        # -----------------------------
        # Prediction & evaluation
        # -----------------------------
        y_pred = model.predict(X_test, num_iteration=model.best_iteration)
        y_test_inv = target_scaler.inverse_transform(y_test.values.reshape(-1,1))
        y_pred_inv = target_scaler.inverse_transform(y_pred.reshape(-1,1))

        global_y_true.append(y_test_inv)
        global_y_pred.append(y_pred_inv)

        mae = mean_absolute_error(y_test_inv, y_pred_inv)
        rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
        mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)
        smape_val = smape(y_test_inv, y_pred_inv)
        r2 = r2_score(y_test_inv, y_pred_inv)
        peak_err = peak_error(y_test_inv, y_pred_inv)

        metrics_df = pd.concat([metrics_df, pd.DataFrame({
            'zone': [zone],
            'MAE': [mae],
            'RMSE': [rmse],
            'MAPE': [mape],
            'SMAPE': [smape_val],
            'R2': [r2],
            'Peak_Error': [peak_err]
        })], ignore_index=True)

        # Save model
        model.save_model(f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/LightGBM_{version}/{zone}.txt')
        print(metrics_df.iloc[-1])
        # Plot predictions
        plt.figure(figsize=(14, 7))
        plt.plot(test_times[:100], y_test_inv[:100], label='Actual', color='orange')
        plt.plot(test_times[:100], y_pred_inv[:100], label='Predicted', color='green')
        plt.xlabel('Date')
        plt.ylabel('Water Consumption')
        plt.title(f'LightGBM Forecast vs Actual - Zone {zone}')
        plt.legend()
        plt.show()
    # -----------------------------
    # Global metrics across all zones
    # -----------------------------
    all_y_true = np.vstack(global_y_true)
    all_y_pred = np.vstack(global_y_pred)

    global_metrics = {
        'MAE': mean_absolute_error(all_y_true, all_y_pred),
        'RMSE': np.sqrt(mean_squared_error(all_y_true, all_y_pred)),
        'MAPE': mean_absolute_percentage_error(all_y_true, all_y_pred),
        'SMAPE': smape(all_y_true, all_y_pred),
        'R2': r2_score(all_y_true, all_y_pred),
        'Peak_Error': peak_error(all_y_true, all_y_pred)
    }

    print("\n✅ Global Metrics:")
    for k,v in global_metrics.items():
        print(f"{k}: {v:.4f}")
    mlflow.log_metrics(global_metrics)

# Save feature/target mapping
with open(f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/LightGBM_{version}/feature_and_target.json', 'w') as f:
    json.dump(feature_and_target, f, indent=4)

print("✅ LightGBM Zone training completed")


Lightgbm (parameteres hypertuning)

In [ ]:
import os, pickle, json
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lightgbm as lgb
import mlflow

# -----------------------------
# Metrics
# -----------------------------
def smape(y_true, y_pred):
    denom = np.abs(y_true)+np.abs(y_pred)
    denom = np.where(denom==0, np.finfo(float).eps, denom)
    return 100/len(y_true)*np.sum(2*np.abs(y_pred-y_true)/denom)

def peak_error(y_true, y_pred, percentile=95):
    peak_val = np.percentile(y_true, percentile)
    peak_idx = y_true>=peak_val
    if np.sum(peak_idx)==0: return np.nan
    return np.mean(np.abs(y_true[peak_idx]-y_pred[peak_idx]))

# -----------------------------
# Paths & parameters
# -----------------------------
root = '/mnt/data/home/zayd/Digital_twin_project/machine_learning/dataset/Ctown/junctions/'
version = '0.0.5_best'
MODEL_DIR = f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/lgbm_model_{version}'
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(f'{MODEL_DIR}/scalers', exist_ok=True)

lag_steps = [1,2,3]
rolling_windows = [3,6,12]

feature_and_target = {}
mlflow.set_experiment("Digital_Twin_Experiments")

with mlflow.start_run(run_name=f"lgbm_model_{version}"):

    for filename in os.listdir(root)[:10]:
        df = pd.read_parquet(os.path.join(root, filename))
        df = df.sort_values(by=["scenario_id", "time_id"]).reset_index(drop=True)
        junction_cols = [col for col in df.columns if col.startswith('J')]

        # Convert from m³/s to L/s
        df[junction_cols] = df[junction_cols] * 1000
        junction = os.path.splitext(filename)[0]

        with mlflow.start_run(run_name=f"{junction}", nested=True):
            # Lag & rolling features
            for lag in lag_steps:
                df[f'{junction}_lag{lag}'] = df[junction].shift(lag)
            for w in rolling_windows:
                df[f'{junction}_rollmean{w}'] = df[junction].rolling(window=w, min_periods=1).mean()
            df = df.fillna(0)

            features = [c for c in df.columns if c != junction]
            target = junction
            feature_and_target[junction] = {"target": target, "features": features, "lags": lag_steps}

            # Scaling
            feature_scaler = MinMaxScaler().fit(df[features])
            target_scaler = MinMaxScaler().fit(df[[target]])
            df[features] = feature_scaler.transform(df[features])
            df[[target]] = target_scaler.transform(df[[target]])

            # Save scalers
            for scaler_name, scaler_obj in zip(['feature','target'], [feature_scaler,target_scaler]):
                path = f'{MODEL_DIR}/scalers/{junction}_{scaler_name}_scaler.save'
                with open(path,'wb') as f: pickle.dump(scaler_obj,f)
                mlflow.log_artifact(path, artifact_path=f"{junction}/scalers")

            X, y = df[features].values, df[target].values
            tscv = TimeSeriesSplit(n_splits=5)

            # LightGBM + hyperparameter tuning
            lgbm = lgb.LGBMRegressor(random_state=42)
            param_grid = {
                'n_estimators': [500, 1000],
                'learning_rate': [0.01, 0.05],
                'num_leaves': [31, 63],
                'max_depth': [-1, 10]
            }
            gsearch = GridSearchCV(
                lgbm, param_grid, cv=tscv,
                scoring='neg_mean_squared_error', verbose=0
            )
            gsearch.fit(X, y)

            # Best params & refit
            best_params = gsearch.best_params_
            best_model = lgb.LGBMRegressor(random_state=42, **best_params)
            best_model.fit(X, y)

            # Evaluate on last fold
            train_idx, test_idx = list(tscv.split(X))[-1]
            X_test, y_test = X[train_idx], y[train_idx]  # correction
            X_test, y_test = X[test_idx], y[test_idx]
            y_pred = best_model.predict(X_test)
            y_test_inv = target_scaler.inverse_transform(y_test.reshape(-1,1))
            y_pred_inv = target_scaler.inverse_transform(y_pred.reshape(-1,1))

            mae = mean_absolute_error(y_test_inv, y_pred_inv)
            rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
            smape_val = smape(y_test_inv, y_pred_inv)
            r2 = r2_score(y_test_inv, y_pred_inv)
            peak_err = peak_error(y_test_inv, y_pred_inv)

            # Log to MLflow
            model_path = f"{MODEL_DIR}/{junction}_{version}.pkl"
            with open(model_path,'wb') as f: pickle.dump(best_model,f)
            mlflow.log_artifact(model_path)

            mlflow.log_params({
                "algo": f"LightGBM_{version}",
                "junction": junction,
                "window": rolling_windows,
                "stride": lag_steps,
                **best_params
            })

            mlflow.log_metrics({
                "MAE": mae, "RMSE": rmse, "SMAPE": smape_val, "R2": r2, "peak_error": peak_err
            })

    # Save feature/target mapping
    feature_target_path = f'{MODEL_DIR}/feature_and_target.json'
    with open(feature_target_path,'w') as f:
        json.dump(feature_and_target, f, indent=4)
    mlflow.log_artifact(feature_target_path)

print("✅ Best LightGBM model per junction logged with MLflow")


XGBoost (single junction)

In [3]:
import os, time, pickle, json, mlflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# -----------------------------
# Custom metrics
# -----------------------------
def smape(y_true, y_pred):
    denom = np.abs(y_true) + np.abs(y_pred)
    denom = np.where(denom == 0, np.finfo(float).eps, denom)
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true)/denom)

def peak_error(y_true, y_pred, percentile=95):
    peak_val = np.percentile(y_true, percentile)
    peak_idx = y_true >= peak_val
    if np.sum(peak_idx) == 0:
        return np.nan
    return np.mean(np.abs(y_true[peak_idx] - y_pred[peak_idx]))

# -----------------------------
# Paths and parameters
# -----------------------------
root = '/mnt/data/home/zayd/Digital_twin_project/machine_learning/dataset/Ctown/junctions/'
version = '0.1.0'
MODEL_DIR = f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/XGBoost_{version}'
os.makedirs(f'{MODEL_DIR}/scalers', exist_ok=True)

lag_steps = [1,2]
rolling_windows = [4,12,24]

mlflow.set_experiment("Digital_Twin_Experiments")  

global_y_true, global_y_pred = [], []
feature_and_target = {}


with mlflow.start_run(run_name=f"XGBoost_{version}"):
    for filename in os.listdir(root)[:10]:
        df = pd.read_parquet(os.path.join(root, filename))
        df = df.sort_values(by=["scenario_id", "time_id"]).reset_index(drop=True)
        junction_cols = [col for col in df.columns if col.startswith('J')]

        # Convert from m³/s to L/s
        df[junction_cols] = df[junction_cols] * 1000
        junction = os.path.splitext(filename)[0]
        with mlflow.start_run(run_name=f"{junction}",nested=True):

            # Create lag features
            for lag in lag_steps:
                df[f'{junction}_lag{lag}'] = df[junction].shift(lag)
            # Create moving average features
            for w in rolling_windows:
                df[f'{junction}_rollmean{w}'] = df[junction].rolling(window=w, min_periods=1).mean()
            df = df.fillna(0)

            features = [c for c in df.columns if c != junction]
            target = junction
            feature_and_target[junction] = {"target": target, "features": features, 'lags': lag_steps}

            # Scaling
            feature_scaler = MinMaxScaler().fit(df[features])
            target_scaler = MinMaxScaler().fit(df[[target]])
            df[features] = feature_scaler.transform(df[features])
            df[[target]] = target_scaler.transform(df[[target]])

            # Save scalers
            with open(f'{MODEL_DIR}/scalers/{junction}_feature_scaler.save', 'wb') as f:
                pickle.dump(feature_scaler, f)
            with open(f'{MODEL_DIR}/scalers/{junction}_target_scaler.save', 'wb') as f:
                pickle.dump(target_scaler, f)

            # Train/test split
            scenario_ids = df['scenario_id'].unique()
            split = int(0.8 * len(scenario_ids))
            train_df = df[df['scenario_id'].isin(scenario_ids[:split])]
            test_df = df[df['scenario_id'].isin(scenario_ids[split:])]

            X_train, y_train = train_df[features], train_df[target]
            X_test, y_test = test_df[features], test_df[target]

            # Train XGBoost
            model = xgb.XGBRegressor(
                n_estimators=5000, learning_rate=0.01, max_depth=6,
                subsample=0.8, colsample_bytree=0.8, objective='reg:squarederror',
                verbosity=1
            )
            start_time = time.time()
            model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
            elapsed_sec = time.time() - start_time
            print(f"{junction} training time: {elapsed_sec:.2f} sec")

            # Predictions
            y_pred = model.predict(X_test)
            y_test_inv = target_scaler.inverse_transform(y_test.values.reshape(-1,1))
            y_pred_inv = target_scaler.inverse_transform(y_pred.reshape(-1,1))
            global_y_true.append(y_test_inv)
            global_y_pred.append(y_pred_inv)

            # -----------------------------
            # MLflow logging
            # -----------------------------
            mlflow.log_params({
                "algo": f"XGBoost_{version}",
                "junction": junction,
                "window": rolling_windows,
                "stride": lag_steps,
                "n_estimators": 5000,
                "learning_rate": 0.01,
                "max_depth": 6,
                "subsample": 0.8,
                "colsample_bytree": 0.8
            })
            mlflow.log_metric("RMSE", np.sqrt(mean_squared_error(y_test_inv, y_pred_inv)))  # RMSE
            mlflow.log_metric("SMAPE", smape(y_test_inv, y_pred_inv))
            mlflow.log_metric("R2_score", r2_score(y_test_inv, y_pred_inv))
            mlflow.log_metric("peak_error", peak_error(y_test_inv, y_pred_inv))

            # Save model
            save_path = os.path.join(MODEL_DIR, f"{junction}.json")
            model.save_model(save_path)
            mlflow.log_artifact(save_path)

# Save feature/target mapping
with open(f'{MODEL_DIR}/feature_and_target.json', 'w') as f:
    json.dump(feature_and_target, f, indent=4)

print("✅ XGBoost training and MLflow logging completed")


J436 training time: 27.38 sec
J27 training time: 26.27 sec
J101 training time: 16.52 sec
J309 training time: 15.01 sec
J280 training time: 22.81 sec
J83 training time: 19.14 sec
J90 training time: 20.03 sec
J303 training time: 26.14 sec
J227 training time: 17.15 sec
J498 training time: 15.20 sec
✅ XGBoost training and MLflow logging completed


z score

In [2]:
import os, time, pickle, json, mlflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# -----------------------------
# Custom metrics
# -----------------------------
def smape(y_true, y_pred):
    denom = np.abs(y_true) + np.abs(y_pred)
    denom = np.where(denom == 0, np.finfo(float).eps, denom)
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true)/denom)

def peak_error(y_true, y_pred, percentile=95):
    peak_val = np.percentile(y_true, percentile)
    peak_idx = y_true >= peak_val
    if np.sum(peak_idx) == 0:
        return np.nan
    return np.mean(np.abs(y_true[peak_idx] - y_pred[peak_idx]))

# -----------------------------
# Paths and parameters
# -----------------------------
root = '/mnt/data/home/zayd/Digital_twin_project/machine_learning/dataset/Ctown/junctions/'
version = '0.1.0_zscore'
MODEL_DIR = f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/XGBoost_{version}'
os.makedirs(f'{MODEL_DIR}/scalers', exist_ok=True)

lag_steps = [1,2]
rolling_windows = [4,12,24]

mlflow.set_experiment("Digital_Twin_Experiments")  

global_y_true, global_y_pred = [], []
feature_and_target = {}

with mlflow.start_run(run_name=f"XGBoost_{version}"):
    for filename in os.listdir(root)[:10]:
        df = pd.read_parquet(os.path.join(root, filename))
        df = df.sort_values(by=["scenario_id", "time_id"]).reset_index(drop=True)
        junction_cols = [col for col in df.columns if col.startswith('J')]

        # Convert from m³/s to L/s
        df[junction_cols] = df[junction_cols] * 1000
        junction = os.path.splitext(filename)[0]
        with mlflow.start_run(run_name=f"{junction}",nested=True):

            # Create lag features
            for lag in lag_steps:
                df[f'{junction}_lag{lag}'] = df[junction].shift(lag)
            # Create moving average features
            for w in rolling_windows:
                df[f'{junction}_rollmean{w}'] = df[junction].rolling(window=w, min_periods=1).mean()
            df = df.fillna(0)

            features = [c for c in df.columns if c != junction]
            target = junction
            feature_and_target[junction] = {"target": target, "features": features, 'lags': lag_steps}

            # -----------------------------
            # Z-score scaling
            # -----------------------------
            feature_scaler = StandardScaler().fit(df[features])
            target_scaler = StandardScaler().fit(df[[target]])
            df[features] = feature_scaler.transform(df[features])
            df[[target]] = target_scaler.transform(df[[target]])

            # Save scalers
            with open(f'{MODEL_DIR}/scalers/{junction}_feature_scaler.save', 'wb') as f:
                pickle.dump(feature_scaler, f)
            with open(f'{MODEL_DIR}/scalers/{junction}_target_scaler.save', 'wb') as f:
                pickle.dump(target_scaler, f)

            # Train/test split
            scenario_ids = df['scenario_id'].unique()
            split = int(0.8 * len(scenario_ids))
            train_df = df[df['scenario_id'].isin(scenario_ids[:split])]
            test_df = df[df['scenario_id'].isin(scenario_ids[split:])]

            X_train, y_train = train_df[features], train_df[target]
            X_test, y_test = test_df[features], test_df[target]

            # Train XGBoost
            model = xgb.XGBRegressor(
                n_estimators=5000, learning_rate=0.01, max_depth=6,
                subsample=0.8, colsample_bytree=0.8, objective='reg:squarederror',
                verbosity=1
            )
            start_time = time.time()
            model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
            elapsed_sec = time.time() - start_time
            print(f"{junction} training time: {elapsed_sec:.2f} sec")

            # Predictions
            y_pred = model.predict(X_test)
            y_test_inv = target_scaler.inverse_transform(y_test.values.reshape(-1,1))
            y_pred_inv = target_scaler.inverse_transform(y_pred.reshape(-1,1))
            global_y_true.append(y_test_inv)
            global_y_pred.append(y_pred_inv)

            # -----------------------------
            # MLflow logging
            # -----------------------------
            mlflow.log_params({
                "algo": f"XGBoost_{version}",
                "junction": junction,
                "window": rolling_windows,
                "stride": lag_steps,
                "n_estimators": 5000,
                "learning_rate": 0.01,
                "max_depth": 6,
                "subsample": 0.8,
                "colsample_bytree": 0.8
            })
            mlflow.log_metric("MAE", mean_absolute_error(y_test_inv, y_pred_inv))
            mlflow.log_metric("RMSE", np.sqrt(mean_squared_error(y_test_inv, y_pred_inv)))
            mlflow.log_metric("SMAPE", smape(y_test_inv, y_pred_inv))
            mlflow.log_metric("R2_score", r2_score(y_test_inv, y_pred_inv))
            mlflow.log_metric("peak_error", peak_error(y_test_inv, y_pred_inv))

            # Save model
            save_path = os.path.join(MODEL_DIR, f"{junction}.json")
            model.save_model(save_path)
            mlflow.log_artifact(save_path)

# Save feature/target mapping
with open(f'{MODEL_DIR}/feature_and_target.json', 'w') as f:
    json.dump(feature_and_target, f, indent=4)

print("✅ XGBoost training and MLflow logging completed")


J436 training time: 18.24 sec
J27 training time: 17.56 sec
J101 training time: 17.64 sec
J309 training time: 16.81 sec
J280 training time: 17.11 sec
J83 training time: 19.58 sec
J90 training time: 17.89 sec
J303 training time: 19.10 sec
J227 training time: 22.62 sec
J498 training time: 22.14 sec
✅ XGBoost training and MLflow logging completed


XGBoost (zones)

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pickle
import os
import mlflow
import json
import time

# -----------------------------
# Metrics
# -----------------------------
def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_true = np.where(y_true == 0, np.finfo(float).eps, y_true)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def smape(y_true, y_pred):
    denominator = np.abs(y_true) + np.abs(y_pred)
    denominator = np.where(denominator == 0, np.finfo(float).eps, denominator)
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / denominator)

def peak_error(y_true, y_pred, percentile=95):
    peak_value = np.percentile(y_true, percentile)
    peak_indices = y_true >= peak_value
    if np.sum(peak_indices) == 0:
        return np.nan
    return np.mean(np.abs(y_true[peak_indices] - y_pred[peak_indices]))

# -----------------------------
# Paths and parameters
# -----------------------------
root = '/mnt/data/home/zayd/Digital_twin_project/machine_learning/dataset/zones_v2/'
version = '0.0.5_zscore---'
os.makedirs(f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/XGBoost_{version}', exist_ok=True)
mlflow.set_tracking_uri("file:model_trained/mlruns")

metrics_df = pd.DataFrame(columns=['zone', 'MAE', 'RMSE', 'MAPE', 'SMAPE', 'R2', 'Peak_Error'])
feature_and_target = {}

lag_steps = [1,2]
rolling_windows = [4,12,24]

global_y_true, global_y_pred = [], []

with mlflow.start_run(run_name=f"XGBoost_{version}"):

    for filename in os.listdir(root)[:1]:
        df = pd.read_parquet(os.path.join(root, filename))
        zone_name = os.path.splitext(filename)[0]
        print(f"Processing zone: {zone_name}")

        junction_cols = [col for col in df.columns if 'demand' in col]

        # -----------------------------
        # Create lag and rolling features
        # -----------------------------
        for lag in lag_steps:
            for junc in junction_cols:
                df[f'{junc}_lag{lag}'] = df[junc].shift(lag)

        for w in rolling_windows:
            for junc in junction_cols:
                df[f'{junc}_rollmean{w}'] = df[junc].rolling(window=w, min_periods=1).mean()

        df = df.fillna(0)

        # -----------------------------
        # Z-score scaling
        # -----------------------------
        numeric_cols = [c for c in df.columns if c not in ['time_id','scenario_id']]
        scalers = {}
        for col in numeric_cols:
            scaler = StandardScaler()
            df[[col]] = scaler.fit_transform(df[[col]])
            scalers[col] = scaler

        # Save scalers
        os.makedirs(f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/XGBoost_{version}/scalers/{zone_name}', exist_ok=True)
        for col, scaler in scalers.items():
            with open(f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/XGBoost_{version}/scalers/{zone_name}/{col}_scaler.save','wb') as f:
                pickle.dump(scaler,f)

        # -----------------------------
        # Train/test split by day
        # -----------------------------
        unique_days = df['day'].unique()
        split = int(0.8 * len(unique_days))
        train_days = unique_days[:split]
        test_days = unique_days[split:]

        train_df = df[df['day'].isin(train_days)]
        test_df = df[df['day'].isin(test_days)]

        for target in junction_cols:
            features = [c for c in numeric_cols if c != target]

            X_train, y_train = train_df[features], train_df[target]
            X_test, y_test = test_df[features], test_df[target]

            # -----------------------------
            # Train XGBoost
            # -----------------------------
            model = xgb.XGBRegressor(
                n_estimators=5000,
                learning_rate=0.01,
                max_depth=6,
                subsample=0.8,
                colsample_bytree=0.8,
                objective='reg:squarederror',
                verbosity=0
            )

            start_time = time.time()
            model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=100)
            elapsed_sec = time.time() - start_time
            print(f"{target} training time: {elapsed_sec:.2f} sec")

            # -----------------------------
            # Predict & inverse scale target
            # -----------------------------
            y_pred = model.predict(X_test)
            y_test_inv = scalers[target].inverse_transform(y_test.values.reshape(-1,1))
            y_pred_inv = scalers[target].inverse_transform(y_pred.reshape(-1,1))

            global_y_true.append(y_test_inv)
            global_y_pred.append(y_pred_inv)

            # Metrics
            mae = mean_absolute_error(y_test_inv, y_pred_inv)
            rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
            mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)
            smape_val = smape(y_test_inv, y_pred_inv)
            r2 = r2_score(y_test_inv, y_pred_inv)
            peak_err = peak_error(y_test_inv, y_pred_inv)

            metrics_df = pd.concat([metrics_df, pd.DataFrame({
                'zone':[f"{zone_name}__{target}"],
                'MAE':[mae],
                'RMSE':[rmse],
                'MAPE':[mape],
                'SMAPE':[smape_val],
                'R2':[r2],
                'Peak_Error':[peak_err]
            })], ignore_index=True)

            # Save model
            os.makedirs(f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/XGBoost_{version}/{zone_name}', exist_ok=True)
            model.save_model(f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/XGBoost_{version}/{zone_name}/{target}.json')

            # MLflow logs
            mlflow.log_metric(f"{target}_MAE", mae)
            mlflow.log_metric(f"{target}_RMSE", rmse)
            mlflow.log_metric(f"{target}_MAPE", mape)
            mlflow.log_metric(f"{target}_SMAPE", smape_val)
            mlflow.log_metric(f"{target}_R2", r2)
            mlflow.log_metric(f"{target}_Peak_Error", peak_err)

            # -----------------------------
            # Plot predictions (fixed length issue)
            # -----------------------------
            plt.figure(figsize=(14, 7))
            plt.plot(test_df.index[:100], y_test_inv[:100], label='Actual', color='orange')
            plt.plot(test_df.index[:100], y_pred_inv[:100], label='Predicted', color='green')
            plt.xlabel('Row index')
            plt.ylabel('Water Consumption')
            plt.title(f'{zone_name} | {target} Forecast')
            plt.legend()
            plt.show()

# -----------------------------
# Global metrics across all junctions
# -----------------------------
all_y_true = np.vstack(global_y_true)
all_y_pred = np.vstack(global_y_pred)

global_metrics = {
    'MAE': mean_absolute_error(all_y_true, all_y_pred),
    'RMSE': np.sqrt(mean_squared_error(all_y_true, all_y_pred)),
    'MAPE': mean_absolute_percentage_error(all_y_true, all_y_pred),
    'SMAPE': smape(all_y_true, all_y_pred),
    'R2': r2_score(all_y_true, all_y_pred),
    'Peak_Error': peak_error(all_y_true, all_y_pred)
}

print("\n✅ Global Metrics:")
for k,v in global_metrics.items():
    print(f"{k}: {v:.4f}")
mlflow.log_metrics(global_metrics)

# Save feature/target mapping
with open(f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/XGBoost_{version}/feature_and_target.json','w') as f:
    json.dump(feature_and_target, f, indent=4)

print("✅ XGBoost training for zones completed")


stacking ensemble (single junction)

In [1]:
import os, time, pickle, json
import pandas as pd
import numpy as np
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb
import mlflow

# -----------------------------
# Metrics
# -----------------------------
def smape(y_true, y_pred):
    denominator = np.abs(y_true)+np.abs(y_pred)
    denominator = np.where(denominator==0, np.finfo(float).eps, denominator)
    return 100/len(y_true)*np.sum(2*np.abs(y_pred-y_true)/denominator)

def peak_error(y_true, y_pred, percentile=95):
    peak_val = np.percentile(y_true, percentile)
    peak_idx = y_true>=peak_val
    if np.sum(peak_idx)==0: return np.nan
    return np.mean(np.abs(y_true[peak_idx]-y_pred[peak_idx]))

# -----------------------------
# Paths & parameters
# -----------------------------
root = '/mnt/data/home/zayd/Digital_twin_project/machine_learning/dataset/Ctown/junctions/'
version = '0.1.0'
MODEL_DIR = f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/stacking_model_{version}'
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(f'{MODEL_DIR}/scalers', exist_ok=True)

lag_steps = [1,2,3]
rolling_windows = [3,6,12]

metrics_df = pd.DataFrame(columns=['junction','MAE','RMSE','SMAPE','R2','Peak_Error'])
feature_and_target = {}
global_y_true, global_y_pred = [], []

mlflow.set_experiment("Digital_Twin_Experiments")

# -----------------------------
# Parent MLflow run
# -----------------------------
with mlflow.start_run(run_name=f"stacking_model_{version}"):

    for filename in os.listdir(root)[:10]:
        df = pd.read_parquet(os.path.join(root, filename))
        df = df.sort_values(by=["scenario_id", "time_id"]).reset_index(drop=True)
        junction_cols = [col for col in df.columns if col.startswith('J')]

        # Convert from m³/s to L/s
        df[junction_cols] = df[junction_cols] * 1000
        junction = os.path.splitext(filename)[0]
        print(f"\n=== Processing junction: {junction} ===")
        

        with mlflow.start_run(run_name=f"{junction}", nested=True):

            # -----------------------------
            # Lag & rolling features
            # -----------------------------
            for lag in lag_steps:
                df[f'{junction}_lag{lag}'] = df[junction].shift(lag)
            for w in rolling_windows:
                df[f'{junction}_rollmean{w}'] = df[junction].rolling(window=w, min_periods=1).mean()
            df = df.fillna(0)

            features = [c for c in df.columns if c!=junction]
            target = junction
            feature_and_target[junction] = {"target":target, "features":features, "lags":lag_steps}

            # -----------------------------
            # Scaling
            # -----------------------------
            feature_scaler = MinMaxScaler().fit(df[features])
            target_scaler = MinMaxScaler().fit(df[[target]])
            df[features] = feature_scaler.transform(df[features])
            df[[target]] = target_scaler.transform(df[[target]])

            # Save scalers
            with open(f'{MODEL_DIR}/scalers/{junction}_feature_scaler.save','wb') as f: pickle.dump(feature_scaler,f)
            with open(f'{MODEL_DIR}/scalers/{junction}_target_scaler.save','wb') as f: pickle.dump(target_scaler,f)
            mlflow.log_artifact(f'{MODEL_DIR}/scalers/{junction}_feature_scaler.save', artifact_path=f"{junction}/scalers")
            mlflow.log_artifact(f'{MODEL_DIR}/scalers/{junction}_target_scaler.save', artifact_path=f"{junction}/scalers")

            # -----------------------------
            # Train/test split by time
            # -----------------------------
            scenario_ids = df['scenario_id'].unique()
            split = int(0.8 * len(scenario_ids))
            train_scenarios = scenario_ids[:split]
            test_scenarios = scenario_ids[split:]

            train_df = df[df['scenario_id'].isin(train_scenarios)]
            test_df = df[df['scenario_id'].isin(test_scenarios)]

            # Extract X and y
            X_train = train_df[features]
            y_train = train_df[target]
            X_test = test_df[features]
            y_test = test_df[target]

            # -----------------------------
            # Stacking Regressor
            # -----------------------------
            estimators = [
                ('xgb', xgb.XGBRegressor(
                    n_estimators=1000, learning_rate=0.01, max_depth=6, random_state=42, verbose=1
                )),
                ('lgbm', lgb.LGBMRegressor(
                    n_estimators=1000, learning_rate=0.01, num_leaves=31, random_state=42, verbose=1
                ))
            ]
            stacking_model = StackingRegressor(estimators=estimators, final_estimator=Ridge(), cv=5)

            print(f"⏳ Training stacking model for junction {junction}...")
            start_time = time.time()
            stacking_model.fit(X_train, y_train)
            elapsed_sec = time.time() - start_time
            print(f"✅ Finished training {junction} in {elapsed_sec:.2f} sec")
            mlflow.log_param(f"{junction}_training_time_sec", elapsed_sec)

            # -----------------------------
            # Predictions & metrics
            # -----------------------------
            y_pred = stacking_model.predict(X_test)
            y_test_inv = target_scaler.inverse_transform(y_test.values.reshape(-1,1))
            y_pred_inv = target_scaler.inverse_transform(y_pred.reshape(-1,1))
            global_y_true.append(y_test_inv)
            global_y_pred.append(y_pred_inv)

            mae = mean_absolute_error(y_test_inv, y_pred_inv)
            rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
            smape_val = smape(y_test_inv, y_pred_inv)
            r2 = r2_score(y_test_inv, y_pred_inv)
            peak_err = peak_error(y_test_inv, y_pred_inv)

            mlflow.log_params({
                "algo": f"Stacking_ensemble_{version}",
                "junction": junction,
                "window": rolling_windows,
                "stride": lag_steps,
                "n_estimators_xgb": 1000,
                "learning_rate_xgb": 0.01,
                "max_depth_xgb": 6,
                "subsample_xgb": 0.8,
                "colsample_bytree_xgb": 0.8,
                "n_estimators_lgbm": 1000,
                "learning_rate_lgbm": 0.01,
                "num_leaves_lgbm": 31,
                "training_time_sec": elapsed_sec
            })

            mlflow.log_metrics({
                'MAE':mae,'RMSE':rmse,'SMAPE':smape_val,'R2_score':r2,'peak_error':peak_err
            })

            # Save model
            model_path = f"{MODEL_DIR}/{junction}.pkl"
            with open(model_path,'wb') as f: pickle.dump(stacking_model,f)
            mlflow.log_artifact(model_path)

    # -----------------------------
    # Global metrics
    # -----------------------------
    all_y_true = np.vstack(global_y_true)
    all_y_pred = np.vstack(global_y_pred)
    global_metrics = {
        'MAE': mean_absolute_error(all_y_true, all_y_pred),
        'RMSE': np.sqrt(mean_squared_error(all_y_true, all_y_pred)),
        'SMAPE': smape(all_y_true, all_y_pred),
        'R2_score': r2_score(all_y_true, all_y_pred),
        'peak_error': peak_error(all_y_true, all_y_pred)
    }
    mlflow.log_metrics(global_metrics)

    # Save feature/target mapping
    with open(f'{MODEL_DIR}/feature_and_target.json','w') as f: json.dump(feature_and_target,f,indent=4)
    mlflow.log_artifact(f'{MODEL_DIR}/feature_and_target.json')

print("✅ Stacking ensemble training completed with MLflow logging")


/home/zayd/.config/matplotlib is not a writable directory
Matplotlib created a temporary cache directory at /tmp/matplotlib-9uyrbmyf because there was an issue with the default path (/home/zayd/.config/matplotlib); it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.



=== Processing junction: J436 ===
⏳ Training stacking model for junction J436...


/home/zayd/Desktop/Digital_twin_project/digitaltwin/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [12:44:00] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007786 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2574
[LightGBM] [Info] Number of data points in the train set: 19200, number of used features: 11
[LightGBM] [Info] Start training from score 0.275829


/home/zayd/Desktop/Digital_twin_project/digitaltwin/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [12:44:24] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/zayd/Desktop/Digital_twin_project/digitaltwin/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [12:44:36] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/zayd/Desktop/Digital_twin_project/digitaltwin/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [12:44:51] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/zayd/Desktop/Digital_twin_project/digitaltwin/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [12:45:06] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain,

...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000237 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2829
[LightGBM] [Info] Number of data points in the train set: 15360, number of used features: 12
[LightGBM] [Info] Start training from score 0.295464
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000226 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2829
[LightGBM] [Info] Number of data points in the train set: 15360, number of used features: 12
[LightGBM] [Info] Start training from score 0.293069
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000278 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enoug

/home/zayd/Desktop/Digital_twin_project/digitaltwin/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [12:54:22] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002179 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2574
[LightGBM] [Info] Number of data points in the train set: 19200, number of used features: 11
[LightGBM] [Info] Start training from score 0.325759


/home/zayd/Desktop/Digital_twin_project/digitaltwin/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [12:54:29] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/zayd/Desktop/Digital_twin_project/digitaltwin/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [12:54:34] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/zayd/Desktop/Digital_twin_project/digitaltwin/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [12:54:38] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/zayd/Desktop/Digital_twin_project/digitaltwin/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [12:54:42] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain,

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008341 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2574
[LightGBM] [Info] Number of data points in the train set: 15360, number of used features: 11
[LightGBM] [Info] Start training from score 0.326460
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000535 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2574
[LightGBM] [Info] Number of data points in the train set: 15360, number of used features: 11
[LightGBM] [Info] Start training from score 0.324822
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003675 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bi

stacking ensemble (Cross validation + hyperparametere_tuning)

In [ ]:
import os, pickle, json
import pandas as pd
import numpy as np
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb
import mlflow

# -----------------------------
# Metrics
# -----------------------------
def smape(y_true, y_pred):
    denom = np.abs(y_true)+np.abs(y_pred)
    denom = np.where(denom==0, np.finfo(float).eps, denom)
    return 100/len(y_true)*np.sum(2*np.abs(y_pred-y_true)/denom)

def peak_error(y_true, y_pred, percentile=95):
    peak_val = np.percentile(y_true, percentile)
    peak_idx = y_true>=peak_val
    if np.sum(peak_idx)==0: return np.nan
    return np.mean(np.abs(y_true[peak_idx]-y_pred[peak_idx]))

# -----------------------------
# Paths & parameters
# -----------------------------
root = '/mnt/data/home/zayd/Digital_twin_project/machine_learning/dataset/Ctown/junctions/'
version = '0.0.5_best'
MODEL_DIR = f'/mnt/data/home/zayd/Digital_twin_project/machine_learning/model_trained/stacking_model_{version}'
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(f'{MODEL_DIR}/scalers', exist_ok=True)

lag_steps = [1,2,3]
rolling_windows = [3,6,12]

feature_and_target = {}
mlflow.set_experiment("Digital_Twin_Experiments")

with mlflow.start_run(run_name=f"stacking_model_{version}"):

    for filename in os.listdir(root):
        df = pd.read_parquet(os.path.join(root, filename))
        df = df.sort_values(by=["scenario_id", "time_id"]).reset_index(drop=True)
        junction_cols = [col for col in df.columns if col.startswith('J')]

        # Convert from m³/s to L/s
        df[junction_cols] = df[junction_cols] * 1000
        junction = os.path.splitext(filename)[0]

        with mlflow.start_run(run_name=f"{junction}", nested=True):
            # Lag & rolling features
            for lag in lag_steps:
                df[f'{junction}_lag{lag}'] = df[junction].shift(lag)
            for w in rolling_windows:
                df[f'{junction}_rollmean{w}'] = df[junction].rolling(window=w, min_periods=1).mean()
            df = df.fillna(0)

            features = [c for c in df.columns if c != junction]
            target = junction
            feature_and_target[junction] = {"target": target, "features": features, "lags": lag_steps}

            # Scaling
            feature_scaler = MinMaxScaler().fit(df[features])
            target_scaler = MinMaxScaler().fit(df[[target]])
            df[features] = feature_scaler.transform(df[features])
            df[[target]] = target_scaler.transform(df[[target]])

            # Save scalers
            for scaler_name, scaler_obj in zip(['feature','target'], [feature_scaler,target_scaler]):
                path = f'{MODEL_DIR}/scalers/{junction}_{scaler_name}_scaler.save'
                with open(path,'wb') as f: pickle.dump(scaler_obj,f)
                mlflow.log_artifact(path, artifact_path=f"{junction}/scalers")

            X, y = df[features].values, df[target].values
            tscv = TimeSeriesSplit(n_splits=5)

            # Stacking + hyperparameter tuning
            estimators = [
                ('xgb', xgb.XGBRegressor(random_state=42, verbosity=1)), 
                ('lgbm', lgb.LGBMRegressor(random_state=42))
            ]
            stacking_model = StackingRegressor(estimators=estimators, final_estimator=Ridge(), cv=5)
            param_grid = {
                'xgb__n_estimators': [500, 1000],
                'xgb__learning_rate': [0.01, 0.05],
                'lgbm__n_estimators': [500, 1000],
                'lgbm__learning_rate': [0.01, 0.05]
            }
            gsearch = GridSearchCV(
                stacking_model, param_grid, cv=tscv,
                scoring='neg_mean_squared_error', verbose=0
            )
            gsearch.fit(X, y)

            # Best params
            best_model_params = gsearch.best_params_
            xgb_params = {k.replace("xgb__", ""): v for k, v in best_model_params.items() if k.startswith("xgb__")}
            lgbm_params = {k.replace("lgbm__", ""): v for k, v in best_model_params.items() if k.startswith("lgbm__")}

            # Refit with best params
            estimators_refit = [
                ('xgb', xgb.XGBRegressor(random_state=42, verbosity=1, **xgb_params)),
                ('lgbm', lgb.LGBMRegressor(random_state=42, **lgbm_params))
            ]
            model = StackingRegressor(estimators=estimators_refit, final_estimator=Ridge())
            model.fit(X, y)

            # Evaluate on last fold
            train_idx, test_idx = list(tscv.split(X))[-1]
            X_test, y_test = X[test_idx], y[test_idx]
            y_pred = model.predict(X_test)
            y_test_inv = target_scaler.inverse_transform(y_test.reshape(-1,1))
            y_pred_inv = target_scaler.inverse_transform(y_pred.reshape(-1,1))

            mae = mean_absolute_error(y_test_inv, y_pred_inv)
            rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
            smape_val = smape(y_test_inv, y_pred_inv)
            r2 = r2_score(y_test_inv, y_pred_inv)
            peak_err = peak_error(y_test_inv, y_pred_inv)

            # Log to MLflow
            model_path = f"{MODEL_DIR}/{junction}_{version}.pkl"
            with open(model_path,'wb') as f: pickle.dump(model,f)
            mlflow.log_artifact(model_path)

            mlflow.log_params({
                "algo": f"Stacking_{version}",
                "junction": junction,
                "window": rolling_windows,
                "stride": lag_steps,
            })

            mlflow.log_metrics({
                "MAE": mae, "RMSE": rmse, "SMAPE": smape_val, "R2": r2, "peak_error": peak_err
            })

    # Save feature/target mapping
    feature_target_path = f'{MODEL_DIR}/feature_and_target.json'
    with open(feature_target_path,'w') as f:
        json.dump(feature_and_target, f, indent=4)
    mlflow.log_artifact(feature_target_path)

print("✅ Best stacking model per junction logged with MLflow")


Prophet (single junction)


In [6]:
import os
os.environ["TMPDIR"] = "/mnt/data/home/zayd/tmp"
os.environ["TEMP"] = "/mnt/data/home/zayd/tmp"
os.environ["TMP"] = "/mnt/data/home/zayd/tmp"

import os, time, pickle, json
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import mlflow

# -----------------------------
# Metrics
# -----------------------------
def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.where(y_true==0, np.finfo(float).eps, y_true)
    return np.mean(np.abs((y_true - y_pred)/y_true))*100

def smape(y_true, y_pred):
    denominator = np.abs(y_true) + np.abs(y_pred)
    denominator = np.where(denominator==0, np.finfo(float).eps, denominator)
    return 100/len(y_true) * np.sum(2*np.abs(y_pred-y_true)/denominator)

def peak_error(y_true, y_pred, percentile=95):
    peak_val = np.percentile(y_true, percentile)
    peak_idx = y_true >= peak_val
    if np.sum(peak_idx)==0: return np.nan
    return np.mean(np.abs(y_true[peak_idx]-y_pred[peak_idx]))

# -----------------------------
# Paths & parameters
# -----------------------------
root = '/mnt/data/home/zayd/Digital_twin_project/machine_learning/dataset/Ctown/junctions/'
version = '0.1.0'
MODEL_DIR = f'./model_trained/Prophet_{version}'
os.makedirs(MODEL_DIR, exist_ok=True)

metrics_df = pd.DataFrame(columns=['junction','MAE','RMSE','SMAPE','R2','Peak_Error'])
feature_and_target = {}

mlflow.set_experiment("Digital_Twin_Experiments")

# -----------------------------
# Parent MLflow run
# -----------------------------
with mlflow.start_run(run_name=f"Prophet_{version}"):

    for filename in os.listdir(root)[:10]:
        df = pd.read_parquet(os.path.join(root, filename))
        df = df.sort_values(by=["scenario_id", "time_id"]).reset_index(drop=True)
        junction_cols = [col for col in df.columns if col.startswith('J')]

        # Convert from m³/s to L/s
        df[junction_cols] = df[junction_cols] * 1000
        junction = os.path.splitext(filename)[0]

        with mlflow.start_run(run_name=f"{junction}", nested=True):
            # -----------------------------
            # Scenario-based split
            # -----------------------------
            scenario_ids = df['scenario_id'].unique()
            split_idx = int(0.8*len(scenario_ids))
            train_df = df[df['scenario_id'].isin(scenario_ids[:split_idx])]
            test_df = df[df['scenario_id'].isin(scenario_ids[split_idx:])]

            # Prepare Prophet data
            train_prophet = train_df[['time_id', junction]].rename(columns={'time_id':'ds', junction:'y'}).fillna(method='ffill')
            test_prophet = test_df[['time_id', junction]].rename(columns={'time_id':'ds', junction:'y'}).fillna(method='ffill')
            feature_and_target[junction] = {"target": junction}

            # -----------------------------
            # Train Prophet
            # -----------------------------
            model = Prophet(daily_seasonality=False)
            start_time = time.time()
            model.fit(train_prophet)
            elapsed_sec = time.time()-start_time

            # Log detailed parameters
            mlflow.log_params({
                "algo": f"Prophet_{version}",
                "daily_seasonality": False,
                "yearly_seasonality": model.yearly_seasonality,
                "weekly_seasonality": model.weekly_seasonality,
                "holidays": bool(model.holidays),
                "changepoint_prior_scale": model.changepoint_prior_scale,
                "seasonality_prior_scale": model.seasonality_prior_scale,
                "training_time_sec": elapsed_sec
            })

            # Forecast
            future = test_prophet[['ds']]
            forecast = model.predict(future)
            y_pred = forecast['yhat'].values
            y_true = test_prophet['y'].values

            # Metrics
            mae = mean_absolute_error(y_true, y_pred)
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            mape = mean_absolute_percentage_error(y_true, y_pred)
            smape_val = smape(y_true, y_pred)
            r2 = r2_score(y_true, y_pred)
            peak_err = peak_error(y_true, y_pred)

            # Log metrics for this junction
            mlflow.log_metrics({
                'MAE': mae,
                'RMSE': rmse,
                # 'MAPE': mape,
                'SMAPE': smape_val,
                'R2_score': r2,
                'peak_error': peak_err
            })

            # Save model
            model_path = os.path.join(MODEL_DIR, f"{junction}.pkl")
            with open(model_path,'wb') as f: pickle.dump(model,f)
            mlflow.log_artifact(model_path)

    # Save feature/target mapping
    with open(os.path.join(MODEL_DIR, 'feature_and_target.json'),'w') as f: json.dump(feature_and_target,f,indent=4)
    mlflow.log_artifact(os.path.join(MODEL_DIR,'feature_and_target.json'))

print("✅ Prophet training with detailed parameter logging completed")


/home/zayd/Desktop/Digital_twin_project/digitaltwin/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_11861/2692068839.py:68: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_prophet = train_df[['time_id', junction]].rename(columns={'time_id':'ds', junction:'y'}).fillna(method='ffill')
/tmp/ipykernel_11861/2692068839.py:69: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_prophet = test_df[['time_id', junction]].rename(columns={'time_id':'ds', junction:'y'}).fillna(method='ffill')
13:09:17 - cmdstanpy - INFO - Chain [1] start processing
13:09:19 - cmdstanpy - INFO - Chain [1] done processing
/tmp/ipykerne

✅ Prophet training with detailed parameter logging completed


Prophet (zones)

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os
import pickle
import mlflow
import json
import time

# -----------------------------
# Metrics
# -----------------------------
def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.where(y_true == 0, np.finfo(float).eps, y_true)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def smape(y_true, y_pred):
    denominator = np.abs(y_true) + np.abs(y_pred)
    denominator = np.where(denominator == 0, np.finfo(float).eps, denominator)
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / denominator)

def peak_error(y_true, y_pred, percentile=95):
    peak_value = np.percentile(y_true, percentile)
    peak_indices = y_true >= peak_value
    if np.sum(peak_indices) == 0:
        return np.nan
    return np.mean(np.abs(y_true[peak_indices] - y_pred[peak_indices]))

# -----------------------------
# Paths and parameters
# -----------------------------
root = '/mnt/data/home/zayd/Digital_twin_project/machine_learning/dataset/junctions_new/'
version = '0.0.1'
os.makedirs(f'./model_trained/Prophet_{version}', exist_ok=True)
mlflow.set_tracking_uri("file:model_trained/mlruns")

metrics_df = pd.DataFrame(columns=['junction', 'MAE', 'RMSE', 'MAPE', 'SMAPE', 'R2', 'Peak_Error'])
feature_and_target = {}

global_y_true, global_y_pred = [], []

with mlflow.start_run(run_name=f"Prophet_{version}"):

    for filename in os.listdir(root)[:70]:
        df = pd.read_parquet(os.path.join(root, filename))
        junction = os.path.splitext(filename)[0]

        # -----------------------------
        # Scenario-based train/test split
        # -----------------------------
        scenario_ids = df['scenario_id'].unique()
        split_idx = int(0.8 * len(scenario_ids))
        train_scenarios = scenario_ids[:split_idx]
        test_scenarios = scenario_ids[split_idx:]

        train_df = df[df['scenario_id'].isin(train_scenarios)]
        test_df = df[df['scenario_id'].isin(test_scenarios)]

        # Prepare for Prophet
        train_prophet = train_df[['time_id', junction]].rename(columns={'time_id': 'ds', junction: 'y'})
        test_prophet = test_df[['time_id', junction]].rename(columns={'time_id': 'ds', junction: 'y'})
        train_prophet['y'] = train_prophet['y'].fillna(method='ffill')
        test_prophet['y'] = test_prophet['y'].fillna(method='ffill')

        feature_and_target[junction] = {"target": junction}

        # -----------------------------
        # Train Prophet
        # -----------------------------
        model = Prophet(daily_seasonality=False)
        start_time = time.time()
        model.fit(train_prophet)
        elapsed_sec = time.time() - start_time
        print(f"{junction} training time: {elapsed_sec:.2f} sec")

        # -----------------------------
        # Forecast test scenarios
        # -----------------------------
        future = test_prophet[['ds']]
        forecast = model.predict(future)
        y_pred = forecast['yhat'].values
        y_true = test_prophet['y'].values

        global_y_true.append(y_true)
        global_y_pred.append(y_pred)

        # -----------------------------
        # Metrics per junction
        # -----------------------------
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mape = mean_absolute_percentage_error(y_true, y_pred)
        smape_val = smape(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        peak_err = peak_error(y_true, y_pred)

        metrics_df = pd.concat([metrics_df, pd.DataFrame({
            'junction': [junction],
            'MAE': [mae],
            'RMSE': [rmse],
            'MAPE': [mape],
            'SMAPE': [smape_val],
            'R2': [r2],
            'Peak_Error': [peak_err]
        })], ignore_index=True)

        # Save model
        with open(f'model_trained/Prophet_{version}/{junction}.pkl', 'wb') as f:
            pickle.dump(model,f)

    # -----------------------------
    # Global metrics
    # -----------------------------
    all_y_true = np.concatenate(global_y_true)
    all_y_pred = np.concatenate(global_y_pred)

    global_metrics = {
        'MAE': mean_absolute_error(all_y_true, all_y_pred),
        'RMSE': np.sqrt(mean_squared_error(all_y_true, all_y_pred)),
        'MAPE': mean_absolute_percentage_error(all_y_true, all_y_pred),
        'SMAPE': smape(all_y_true, all_y_pred),
        'R2': r2_score(all_y_true, all_y_pred),
        'Peak_Error': peak_error(all_y_true, all_y_pred)
    }

    print("\n✅ Global Metrics:")
    for k,v in global_metrics.items():
        print(f"{k}: {v:.4f}")
    mlflow.log_metrics(global_metrics)

# Save feature/target mapping
with open(f'model_trained/Prophet_{version}/feature_and_target.json', 'w') as f:
    json.dump(feature_and_target, f, indent=4)

print("✅ Prophet training completed")
